# How do I _create_, _monitor_, and get task _results_?
### Overview
We are getting into advanced techniques here and will need to leverage a few other cookbooks. You will need an app and some files in your project, then it is easy to start one. Unlike the other cookbooks, we will only show the object-based approach to do this.

### Prerequisites
 1. You need to be a member (or owner) of _at least **one**_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="set_AUTH_TOKEN.ipynb">**set_AUTH_TOKEN.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. You understand how to <a href="apps_copyFromPublicApps.ipynb" target="_blank">add public apps </a> (we will just use that call directly and pick one here).
 5. You understand how to <a href="files_copyFromPublicReference.ipynb" target="_blank">add public files</a>  (we will show you the exact query).
 6. You may want to review how to <a href="apps_detailOne.ipynb" target="_blank"> get details </a> of your app (we will assume you do, and pass the appropriate inputs).
 
### Imports and Definitions
A single call is sufficient to create a task. All methods and objects are defined in the apimethods.py file. Here there is **no advantage** in using the object to POST the call, but it is rather useful searching the results of the GET calls.

In [ ]:
from defs.apimethods import *

## (Option A) Use API() _object_
We will first list all our projects, then add a _Public Reference App_<sup>1</sup> and _Public Reference File_ to the first project. We will then create a task and have it start immediately.

<sup>1</sup> Here we also explicitly set _'limit':100_ inside the _query_. This helps speed up the auto-pagination feature within the object constructor.

### Step 1) Get the App

In [ ]:
# [USER INPUT] Set project (p_) indices:
p_index_target = 0                                # target project
a_name = 'Picard CreateSequenceDictionary'

# LIST all projects
existing_projects = API('projects')         

# LIST all apps in project
my_apps = API(path='apps', query={'project': existing_projects.id[p_index_target]})

# ALTERNATIVE way to LIST all Apps using VISIBILITY and searching by NAME
apps_to_copy = API(path='apps', query={'visibility': 'public', 'limit': 100})
a_index = apps_to_copy.name.index(a_name)

# Check if first file already exists in the second project
if apps_to_copy.id[a_index] in my_apps.id:
    print('App already exists in second project, please try another app')
else:
    print('App (%s) does not exist in Project (%s); copying now' % \
          (apps_to_copy.name[a_index], existing_projects.id[p_index_target]))
    
    # COPY the selected file from first to second project
    API(path=('apps/' + apps_to_copy.id[a_index] + '/actions/copy'), \
        method='POST', \
        data={'project': existing_projects.id[p_index_target],\
              'name': apps_to_copy.name[a_index]})

    # re-list the five files in secondProject to verify the copy worked
    my_apps = API(path='apps', query={'project': existing_projects.id[p_index_target]})
    
    if apps_to_copy.name[a_index] in my_apps.name:
        print('Sucessfully copied one app!')
    else:
        print('Something went wrong...')
    my_apps = API(path='apps', query={'project': existing_projects.id[p_index_target]})

### Step 2) Get the File

In [ ]:
# [USER INPUT] Set file and Public Reference Project names here:
f_name = 'ucsc.hg19.fasta'                
source_project = 'admin/sbg-public-data'          # name of Public Files Project

# LIST all projects
existing_projects = API('projects')         

# LIST all files in the source and target project
my_files_source = API(path='files', \
                      query={'project':source_project, \
                            'limit':100})
my_files_target = API(path='files', \
                            query={'project': existing_projects.id[p_index_target]})
f_index = my_files_source.name.index(f_name)

# Check if first file already exists in the target project
if my_files_source.name[f_index] in my_files_target.name:
    print('file already exists in second project, please try another file')
else:
    print('File (%s) does not exist in Project (%s); copying now' % \
          (my_files_source.name[f_index], existing_projects.id[p_index_target]))
    
    # COPY the selected file from source to target project
    API(path=('files/' + my_files_source.id[f_index] + '/actions/copy'), \
        method='POST', \
        data={'project': existing_projects.id[p_index_target],\
              'name': my_files_source.name[f_index]}) 

    # re-list files in target project to verify the copy worked
    my_files_target = API(path='files', \
                          query={'project': existing_projects.id[p_index_target]})
    f_index = my_files_source.name.index(f_name)
    
    if my_files_source.name[f_index] in my_files_target.name:
        print('Sucessfully copied one file!')
        f_index = my_files_target.name.index(f_name)
    else:
        print('Something went wrong...')

### Step 3) Create the task
Here we use the reference file and set one of the 11 optional configuration inputs. Note that input files are passed a _dictionary_ (or a _list_ of _dictionaries_) while configuration parameters are passed just the values.

In [ ]:
# Task description
new_task = {'description': 'APIs are awesome',
    'name': ('task created with task_create.ipynb'),
    'app': (my_apps.id[0]),                                  # App should be at index 0 since we just added it
    'project': existing_projects.id[p_index_target],
    'inputs': {
        'input_reference': {                               # .fasta reference file
            'class': 'File',
            'path': my_files_target.id[f_index],
            'name': my_files_target.name[f_index]
        },
        'quiet': "True"
    }
}

# Create and RUN a task
my_task = API(method='POST', data=new_task, path='tasks/', query = {'action': 'run'})

### Step 4) Print task status
Here we poll the recently created task. 

In [ ]:
my_task = API(method='GET', path=('tasks/' + my_task.id))
print('Your task is in %s status' % (my_task.status))

### Step 5) Wait for task completion
Simple loop to ping for task completion.

In [ ]:
# [USER INPUT] Set loop time (seconds):
loop_time = 120
flag = {'taskRunning': True}

while flag['taskRunning']:
    print('Pinging CGC for task completion, will download summary files once all tasks completed.')
    my_task = API(method='GET', path=('tasks/' + my_task.id))
    if my_task.status == 'COMPLETED':
        flag['taskRunning'] = False
        print('Task has completed, life is beautiful')
    elif my_task.status  == 'FAILED':  
        print('Task failed, can not continue')
        exit()                      # this will not fail gracefully in iPython
    else:
        sleep(loop_time)

### Step 6) Get task outputs
Here we poll the recently created task. 

In [ ]:
my_task = API(method='GET', path=('tasks/' + my_task.id))
print('Your task created %i outputs' % (len(my_task.outputs.keys())))
for f_name in my_task.outputs:
    print(' task output (%s) is the file (%s)' % (f_name, my_task.outputs[f_name]['name']))

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.cancergenomicscloud.org/docs/create-a-new-task) and [here](http://docs.cancergenomicscloud.org/docs/perform-an-action-on-a-specific-task)